In [1]:
import ollama 

dataset = []
with open('cat-facts.txt','r',encoding="utf-8") as file:
    dataset= file.readlines()
    print(f"Loaded {len(dataset)} Entries")

Loaded 150 Entries


In [2]:

EMBEDDING_MODEL = 'hf.co/CompendiumLabs/bge-base-en-v1.5-gguf'
LANGUAGE_MODEL = 'hf.co/bartowski/Llama-3.2-1B-Instruct-GGUF'

In [3]:
VECTOR_DB = []
def add_chunks_to_database(chunks):
    embedding = ollama.embed(model= EMBEDDING_MODEL,input=chunks)['embeddings'][0]
    VECTOR_DB.append((chunks,embedding))

In [4]:
for i ,chunk in enumerate(dataset):
    add_chunks_to_database(chunk)
    print(f"Added {i+1}/{len(dataset)} to database")

Added 1/150 to database
Added 2/150 to database
Added 3/150 to database
Added 4/150 to database
Added 5/150 to database
Added 6/150 to database
Added 7/150 to database
Added 8/150 to database
Added 9/150 to database
Added 10/150 to database
Added 11/150 to database
Added 12/150 to database
Added 13/150 to database
Added 14/150 to database
Added 15/150 to database
Added 16/150 to database
Added 17/150 to database
Added 18/150 to database
Added 19/150 to database
Added 20/150 to database
Added 21/150 to database
Added 22/150 to database
Added 23/150 to database
Added 24/150 to database
Added 25/150 to database
Added 26/150 to database
Added 27/150 to database
Added 28/150 to database
Added 29/150 to database
Added 30/150 to database
Added 31/150 to database
Added 32/150 to database
Added 33/150 to database
Added 34/150 to database
Added 35/150 to database
Added 36/150 to database
Added 37/150 to database
Added 38/150 to database
Added 39/150 to database
Added 40/150 to database
Added 41/

In [5]:
def cosine_similarity(a,b):
    dot_product = sum([x*y for x,y in zip(a,b)])
    norm_a = sum([x**2 for x in a])**0.5
    norm_b = sum([y**2 for y in b])**0.5
    return dot_product /(norm_a * norm_b)


In [6]:
def retrieve(query,top_n=3):
    query_embedding = ollama.embed(model=EMBEDDING_MODEL,input=query)['embeddings'][0]
    similarities =[]
    for chunk,embedding in VECTOR_DB:
        similarity = cosine_similarity(query_embedding,embedding)
        similarities.append((chunk, similarity))
        similarities.sort(key=lambda x:x[1],reverse=True)
        return similarities[:top_n]

In [12]:
# Chatbot input
input_query = input("Ask your question: ")

# Retrieve relevant knowledge
retrieved_knowledge = retrieve(input_query)
print("\nRetrieved knowledge:")
for chunk, similarity in retrieved_knowledge:
    print(f" - (similarity: {similarity:.2f}) {chunk}")


Retrieved knowledge:
 - (similarity: 0.34) On average, cats spend 2/3 of every day sleeping. That means a nine-year-old cat has been awake for only three years of its life.



In [9]:
# Build prompt
context_text = '\n'.join([f' - {chunk.strip()}' for chunk, _ in retrieved_knowledge])
instruction_prompt = f'''You are a helpful chatbot.
Use only the following pieces of context to answer the question. Don't make up any new information:
{context_text}
'''

In [10]:
# Chat with Ollama
stream = ollama.chat(
    model=LANGUAGE_MODEL,
    messages=[
        {'role': 'system', 'content': instruction_prompt},
        {'role': 'user', 'content': input_query}
    ],
    stream=True
)

In [11]:
print("\nChatbot Response:")
for chunk in stream:
    print(chunk['message']['content'], end='', flush=True)


Chatbot Response:
A cat! A small, furry, and often adorable creature that brings joy to many people's lives.